# Security Event Simulator

Trigger security events via the **Mock EDR** and **Mock SIEM** APIs and watch them flow through the event-driven revocation pipeline.

## Event Flow Architecture

```
Mock EDR / SIEM  ──▶  Kafka (security-events)  ──▶  EDA Rulebook  ──▶  Dogtag Revocation
```

## 26 Event Types Across 6 Categories

| Category | Event Types | Count |
|----------|-------------|-------|
| **Original** | malware_detection, credential_theft, ransomware, c2_communication, lateral_movement, privilege_escalation, suspicious_script | 7 |
| **PKI/Cert** | key_compromise, geo_anomaly, compliance_violation, mitm_detected, rogue_ca | 5 |
| **IoT** | firmware_integrity, device_cloning, iot_anomaly, protocol_attack | 4 |
| **Identity** | impossible_travel, service_account_abuse, mfa_bypass, kerberoasting | 4 |
| **Network** | tls_downgrade, ct_log_mismatch, ocsp_bypass | 3 |
| **SIEM** | data_exfiltration, unauthorized_access, certificate_misuse | 3 |

The EDA rulebook has **87 rules** — each event type has explicit RSA/ECC/PQC routing, plus 4 FreeIPA identity rules and 2 logging rules. Every triggered event results in automatic certificate revocation on the matching PKI hierarchy.

## Configuration

| Service | Internal URL | External Port |
|---------|-------------|---------------|
| Mock EDR | `edr.cert-lab.local:8000` | 8082 |
| Mock SIEM | `siem.cert-lab.local:8000` | 8083 |
| Kafka | `kafka.cert-lab.local:9092` | 9092 |

In [ ]:
import os
import json
import time
from datetime import datetime

import httpx
import pandas as pd
from IPython.display import display, clear_output
from kafka import KafkaConsumer

EDR_URL = "http://edr.cert-lab.local:8000"
SIEM_URL = "http://siem.cert-lab.local:8000"
KAFKA_SERVERS = os.getenv("KAFKA_BOOTSTRAP_SERVERS", "kafka.cert-lab.local:9092")
KAFKA_TOPIC = os.getenv("KAFKA_TOPIC", "security-events")

# Category mapping for convenience
CATEGORIES = {
    "original": ["malware_detection", "credential_theft", "ransomware",
                 "c2_communication", "lateral_movement", "privilege_escalation",
                 "suspicious_script"],
    "pki": ["key_compromise", "geo_anomaly", "compliance_violation",
            "mitm_detected", "rogue_ca"],
    "iot": ["firmware_integrity", "device_cloning", "iot_anomaly",
            "protocol_attack"],
    "identity": ["impossible_travel", "service_account_abuse",
                 "mfa_bypass", "kerberoasting"],
    "network": ["tls_downgrade", "ct_log_mismatch", "ocsp_bypass"],
    "siem": ["data_exfiltration", "unauthorized_access", "certificate_misuse"],
}

print(f"EDR: {EDR_URL}")
print(f"SIEM: {SIEM_URL}")
print(f"Kafka: {KAFKA_SERVERS}")

## List All Scenarios

Fetch the full scenario catalog from the Mock EDR. Each scenario maps to one of the 26 event types.

In [ ]:
try:
    resp = httpx.get(f"{EDR_URL}/scenarios", timeout=10)
    resp.raise_for_status()
    scenarios = resp.json()

    if isinstance(scenarios, list):
        df = pd.DataFrame(scenarios)
        print(f"Available scenarios: {len(df)}")
        display(df)
    elif isinstance(scenarios, dict):
        # Some versions return {"scenarios": [...]}
        items = scenarios.get("scenarios", scenarios)
        if isinstance(items, list):
            df = pd.DataFrame(items)
            print(f"Available scenarios: {len(df)}")
            display(df)
        else:
            print(json.dumps(scenarios, indent=2))
except Exception as e:
    print(f"Failed to fetch scenarios: {e}")
    print("Make sure mock-edr is running.")

## Trigger Single Event

Pick a scenario, device ID, severity, and PKI type, then trigger the event via the Mock EDR.

**Modify the variables below** to change what gets triggered.

In [ ]:
# --- Configure the event ---
SCENARIO = "Certificate Private Key Compromise"  # scenario name from catalog
DEVICE_ID = "notebook-device-01"                 # device FQDN prefix
SEVERITY = "critical"                            # critical, high, medium, low
PKI_TYPE = "rsa"                                 # rsa, ecc, pqc

payload = {
    "device_id": DEVICE_ID,
    "scenario": SCENARIO,
    "severity": SEVERITY,
    "pki_type": PKI_TYPE,
}

try:
    resp = httpx.post(f"{EDR_URL}/trigger", json=payload, timeout=10)
    resp.raise_for_status()
    result = resp.json()
    print(f"Event triggered successfully!")
    print(json.dumps(result, indent=2))
except Exception as e:
    print(f"Failed to trigger event: {e}")

## Trigger by Category

Fire all events in a category at once. Change `CATEGORY` to one of: `original`, `pki`, `iot`, `identity`, `network`, `siem`.

In [ ]:
CATEGORY = "iot"         # original, pki, iot, identity, network, siem
CATEGORY_PKI = "rsa"     # rsa, ecc, pqc

event_types = CATEGORIES.get(CATEGORY, [])
if not event_types:
    print(f"Unknown category: {CATEGORY}")
    print(f"Available: {list(CATEGORIES.keys())}")
else:
    print(f"Triggering {len(event_types)} events in category '{CATEGORY}' on {CATEGORY_PKI.upper()} PKI...")
    results = []
    for et in event_types:
        payload = {
            "device_id": f"cat-test-{et[:8]}",
            "scenario": et,
            "severity": "critical",
            "pki_type": CATEGORY_PKI,
        }
        try:
            resp = httpx.post(f"{EDR_URL}/trigger", json=payload, timeout=10)
            results.append({"event_type": et, "status": resp.status_code, "ok": resp.is_success})
        except Exception as e:
            results.append({"event_type": et, "status": "error", "ok": False})

    df = pd.DataFrame(results)
    print(f"\nResults: {sum(df['ok'])}/{len(df)} succeeded")
    display(df)

## SIEM Correlation Rules

Fetch the correlation rules configured in the Mock SIEM.

In [ ]:
try:
    resp = httpx.get(f"{SIEM_URL}/rules", timeout=10)
    resp.raise_for_status()
    rules = resp.json()

    if isinstance(rules, list):
        df = pd.DataFrame(rules)
        print(f"SIEM correlation rules: {len(df)}")
        display(df)
    elif isinstance(rules, dict):
        items = rules.get("rules", rules)
        if isinstance(items, list):
            df = pd.DataFrame(items)
            print(f"SIEM correlation rules: {len(df)}")
            display(df)
        else:
            print(json.dumps(rules, indent=2))
except Exception as e:
    print(f"Failed to fetch SIEM rules: {e}")
    print("Make sure mock-siem is running.")

## Multi-PKI Burst

Trigger the same scenario across **RSA**, **ECC**, and **PQC** simultaneously to test cross-PKI event routing.

In [ ]:
BURST_SCENARIO = "Certificate Private Key Compromise"

results = []
for pki in ["rsa", "ecc", "pqc"]:
    payload = {
        "device_id": f"burst-{pki}",
        "scenario": BURST_SCENARIO,
        "severity": "critical",
        "pki_type": pki,
    }
    try:
        resp = httpx.post(f"{EDR_URL}/trigger", json=payload, timeout=10)
        results.append({
            "pki_type": pki.upper(),
            "status": resp.status_code,
            "ok": resp.is_success,
            "response": resp.json() if resp.is_success else resp.text[:100],
        })
    except Exception as e:
        results.append({"pki_type": pki.upper(), "status": "error", "ok": False, "response": str(e)})

df = pd.DataFrame(results)
print(f"Multi-PKI burst for '{BURST_SCENARIO}':")
display(df[["pki_type", "status", "ok"]])

## Event History

Read recent events from Kafka to confirm what was just triggered. This reads up to 50 recent events from the `security-events` topic.

In [ ]:
try:
    consumer = KafkaConsumer(
        KAFKA_TOPIC,
        bootstrap_servers=KAFKA_SERVERS,
        auto_offset_reset="earliest",
        enable_auto_commit=False,
        consumer_timeout_ms=5000,
        value_deserializer=lambda m: json.loads(m.decode("utf-8")),
    )

    events = []
    for message in consumer:
        events.append(message.value)
    consumer.close()

    # Show last 50
    recent = events[-50:] if len(events) > 50 else events
    print(f"Total events in topic: {len(events)}, showing last {len(recent)}")

    if recent:
        df = pd.DataFrame(recent)
        cols = ["timestamp", "event_type", "severity", "pki_type", "device_fqdn", "certificate_serial"]
        available = [c for c in cols if c in df.columns]
        display(df[available] if available else df)
    else:
        print("No events found.")
except Exception as e:
    print(f"Failed to read Kafka: {e}")